<a href="https://colab.research.google.com/github/sarahwierzbicki/projects/blob/main/predict_cat_on_postings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

In [ ]:
!pip install PyPDF2
!pip install python-docx

pre-processing

In [ ]:
#imports
from PyPDF2 import PdfReader
from docx import Document
import spacy
import numpy as np
import pandas as pd
import re

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/capstone/filtered_jobpostings.csv')

In [ ]:
import preprocessing_module

In [ ]:
df

In [ ]:
job_df = df.drop_duplicates()

In [ ]:
job_df['description'] = job_df['description'].str.strip()

In [ ]:
df['processed_description'] = df['description'].apply(preprocessing_module.process)

In [ ]:
df

predict

In [ ]:
from datasets import Dataset
from transformers import pipeline

In [ ]:
model_name = "/content/drive/MyDrive/capstone/final-roberta-for-resume-classifying"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
model.eval()

In [ ]:
#make dataset
predict_df = df[['processed_description', 'title']]
predict_dataset = Dataset.from_pandas(predict_df)

In [ ]:
#tokenize inputs
def tokenize(batch):
      return tokenizer(batch['processed_description'], padding="max_length", truncation=True)

In [ ]:
predict_dataset = predict_dataset.remove_columns('title')

In [ ]:
dataset = predict_dataset.map(tokenize, batched = True)

In [ ]:
dataset.set_format("torch", columns=["input_ids", "attention_mask"])

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
test_args = TrainingArguments(
    output_dir="./results",
    do_train=False,
    do_predict=True,
    per_device_eval_batch_size=8,
    dataloader_drop_last=False
   )

In [ ]:
trainer = Trainer(
       model=model,
       args=test_args,
   )

In [ ]:
!pip install numpy==1.26.4

In [ ]:
results = trainer.predict(dataset)

In [ ]:
print(results.predictions)

In [ ]:
print(results.metrics)

In [ ]:
#manual mapping :,(
id2label = {0: "ACCOUNTANT", 1: 'ADVOCATE', 2: 'AGRICULTURE', 3: 'APPAREL', 4: 'ARTS', 5: 'AUTOMOBILE', 6: 'AVIATION', 7: 'BANKING', 8: 'BPO', 9: 'BUSNIESS-DEVELOPMENT', 10: 'CHEF', 11: 'CONSTRUCTION', 12: 'CONSULTANT', 13: 'DESIGNER', 14: 'DIGITAL-MEDIA', 15: 'ENGINEERING', 16: 'FINANCE', 17: 'FITNESS', 18: 'HEALTHCARE', 19: 'HR', 20: 'INFORMATION-TECHNOLOGY', 21: 'PUBLIC-RELATIONS', 22: 'SALES', 23: 'TEACHER'}
label2id = {"ACCOUNTANT": 0,'ADVOCATE': 1, 'AGRICULTURE': 2,  'APPAREL':3, 'ARTS': 4, 'AUTOMOBILE': 5, 'AVIATION': 6, 'BANKING': 7, 'BPO': 8, 'BUSNIESS-DEVELOPMENT': 9, 'CHEF': 10, 'CONSTRUCTION': 11, 'CONSULTANT': 12, 'DESIGNER': 13, 'DIGITAL-MEDIA': 14, 'ENGINEERING': 15, 'FINANCE': 16, 'FITNESS': 17, 'HEALTHCARE': 18, 'HR': 19, 'INFORMATION-TECHNOLOGY': 20, 'PUBLIC-RELATIONS': 21, 'SALES': 22, 'TEACHER': 23}

In [ ]:
predicted_indices = np.argmax(results.predictions, axis=1)

In [ ]:
prediction_df = pd.DataFrame(predicted_indices)

In [ ]:
prediction_df['Category'] = prediction_df[0].map(id2label)

In [ ]:
prediction_df

In [ ]:
df

In [ ]:
final_df = prediction_df.join(df)

In [ ]:
final_df.drop('Unnamed: 0', axis = 1, inplace=True)

In [ ]:
final_df

In [ ]:
final_df[final_df['Category'] == 'DIGITAL-MEDIA']

In [ ]:
final_df.to_json('jobpostingswithcategory.json')
final_df.to_csv('jobpostingswithcategory.csv')

In [ ]:
pipe = pipeline(model = '/content/drive/MyDrive/capstone/roberta-for-resume-classifying-infer', task = 'text-classification')

In [ ]:
pipe('passenger service concourse manager summary dedicated enthusiastic leader able motivate employees perform best providing excellent service developing ongoing profitable client relationships creative resourceful flexible able adapt changing priorities maintain positive attitude strong work ethic skills strategic planning team building personable client relationships dedicated flexible relationship building decision making time management conflict resolution adaptability ability work pressure creativity ability work team microsoft word word processing spreadsheet microsoft excel microsoft programs scheduling accomplishments promoted times years diverse skill set managing employees customers areas aviation business proven ability manage complex concourses atlanta airport established trustworthy relationship clients multiple concourses experience current passenger service concourse manager company city state maintain appearance equipment utilized day managing daily operation driving safety policies directly responsible overseeing zone leaders communicated trained pushed operational changes directed maintained healthy working environment working busy atmosphere maintained provided daily schedule dispatchers senior leaders attended delta shift briefing regularly effort communicate expectations day address possible concern raised delta staff addressed overall concourse operation concerns fulfill customer request regularly timely manner provide daily huddle direction staff preparation daily operation coach counsel employees recognize employees diffuse situations challenge delays maintain effective operational performance revamp processes necessary provide leadership employees daily provide hands team work focus drive dress code policy employee professionalism communicate daily client management key positioned employees involved daily operation drive customer satisfaction results cabin service manager company city state managed team man employees managed team leaders directly responsible building day schedule employees responsible coaching counseling documenting employees managed entire concourse ramp communicated managed dispatcher responsible providing dispatcher staff effort turn aircraft day time turn operation responsible managing processes cbp responsible controlling operation failures pushed safety policies regularly cabin service supervisor company city state assistant manager scheduling employees execute directives provided leadership support team leaders assistant aircraft needed continuously provide safety patrol ramp concourse daily held briefing communicate information pasted management train employees team leaders service aircraft cabin service supervisor company city state maintain appearance equipment utilized operations assistant manager scheduling employees execute directives provided leadership support team leaders assistant aircraft needed continuously provide safety patrol ramp concourse daily held briefing communicate information pasted management train employees team leaders service aircraft education training high school diploma academics college park high city state usa academics technical skills months experience cbp seal process admin office work compiling information applications typing submitting applications cbp managing approved rejected applicants handled weekly meeting cbp provided updated based meeting management maintained red black seal count list provided weekly seal list hr management directing employees coming office assistance professionally handling calls directing callers assisting senior managers office needs copying computer input uniform issuing sorting')